
# Motion controllers
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

## Contents
* Writing controller - general
* Writing motor controller - Blender blades
* Pseudomotor basics - Slit controller

### Before writing a new controller
* Before writing a new controller check in the [catalogue](https://github.com/sardana-org/sardana-plugins) if someone already wrote a similar one
* If not, it does not harm to ask on one of the communication channels e.g. mailing list, github issue, etc.

### Controller plugins discovery
* Sardana built-in controllers are importable from `sardana.pool.poolcontrollers`
* The plugin discovery system is based on directory scanning and python module inspection
* Custom controllers should be installed in one of the `PoolPath` directories:
 * Create /controllers directory: `mkdir ~/controllers`
 * In spock: `Pool_demo1_1.put_property({"PoolPath":["/home/vagrant/controllers"]})`
 * Restart the Sardana server
* The path order is important! Controllers in the higher position paths will take precedence over the lower position paths.

### Blender Blades Motor Controller

* Open the code: `kwrite sardana-training/controllers/templates/BlenderBladesMotorCtrl.py`

### Blender Blades Motor Controller

* Install Blender Slits system: `pip3 install --user /home/vagrant/sardana-training/blender-slits`
* Start hardware: `blender-slits-server`
* Blender Blades system (NOT PART OF SARDANA): 
 * Communication protocol is explained in: `sardana-training/blender-slits/README.md`
 * Directions are in the hardware coordinate system
 * Positions are not calibrated in the local coordinate system - beam axis is not the zero!

### [Writing constructor](http://www.sardana-controls.org/devel/howto_controllers/howto_controller.html#constructor)
* Constructor
 * Called on: controllers creation, pool startup and controller's code reload
 * Accepts arguments: instance (name of the controller instance) and properties (dictionary with the controller properties)
 * If an exception is raised when constructing the controller, the controller automatically gets into the Fault state and its status describes the exception that occured
* Instantiate `BlenderBlades` in the constructor

### Instantiate controller
* Deploy controller: `ln -s ~/sardana-training/controllers/templates/BlenderBladesMotorCtrl.py ~/controllers`
* Load it in the system - in spock: `addctrllib BlenderBladesMotorCtrl`
* Create an instance of the controller: `defctrl BlenderBladesMotorController bleblactrl`
* Ask for controllers state: `bleblactrl.state()`

### [Implement StateOne](http://www.sardana-controls.org/devel/howto_controllers/howto_motorcontroller.html#get-motor-state)
* Get axis state (State sequence)
 * Applies only to the to physical elements
 * Called on: state request, during operations e.g. motion, acquisition
 * Returns: state and optionally status, if no status is returned, it will be composed by Sardana from the state (in case of motor also returns limit switches)
 * If an exception is raised when reading the state, the axis automatically gets into the Fault state and the status contains the exception details.

### [Implement StateOne](http://www.sardana-controls.org/devel/howto_controllers/howto_motorcontroller.html#get-motor-state)
* Implement AXIS_NAMES class member for quick lookup to motor identifiers
* Implement AXIS_ID and VALUE class members for refering to the corresponing part of the answer
* Implement method by quering `?state <axis id>`

### [Implement ReadOne](http://www.sardana-controls.org/devel/howto_controllers/howto_motorcontroller.html#get-motor-position)
* Implement method by quering `?pos <axis id>`

### Instantiate top motor
* Reload controller code: `relctrlcls BlenderBladesMotorController` 
* Create motor instance: `defelem top bleblactrl 1`
* Ask for motor state: `top.state()`
* Ask for motor position: `top.position` or `wm top`

### [Implement StartOne](http://www.sardana-controls.org/devel/howto_controllers/howto_motorcontroller.html#move-a-motor)
* Implement method by commanding `<axis id> <pos>`

### Move top motor
* Reload controller code: `relctrlcls BlenderBladesMotorController` 
* Ask motor to move: `umvr top 10`

### [Implement AbortOne](http://www.sardana-controls.org/devel/howto_controllers/howto_motorcontroller.html#abort-a-motor)
* Implement method by commanding `abort`

### Move and abort top motor
* Reload controller code: `relctrlcls BlenderBladesMotorController` 
* Ask motor to move: `umvr top 50`
* Issue `Ctrl+C` in spock

### [Implement standard axis attributes](http://www.sardana-controls.org/devel/howto_controllers/howto_motorcontroller.html#standard-axis-attributes)
* Implement `GetAxisPar` method
 * Implement velocity by querying `?vel <axis id>`
 * Implement acceleration by querying `?acc <axis id>`
 * Implement deceleration by querying `?dec <axis id>`
 * Hardcode base_rate to 0
 * Hardcode steps_per_unit to 1

### Read standard axis attributes
* `top.velocity`
* `top.acceleration`
* `top.deceleration`
* `top.base_rate`
* `top.state_per_unit`

### [Implement standard axis attributes](http://www.sardana-controls.org/devel/howto_controllers/howto_motorcontroller.html#standard-axis-attributes)
* Implement `SetAxisPar` method
 * Implement velocity by commanding `vel <axis id> <value>`
 * Implement acceleration by commanding `acc <axis id> <value>`
 * Implement deceleration by commanding `dec <axis id> <value>`
 * Raise exception when base_rate is set
 * Raise exception when steps_per_unit is set
 
 

### Write standard axis attributes
* `top.velocity = 10`
* `top.acceleration = 0.1`
* `top.deceleration = 0.1`
* `top.base_rate = 5` -> exception!
* `top.state_per_unit = 100` -> exception!
* Make some moves...

### Align slits
* Define the rest of the motors:
 * `defm bottom bleblactrl 2`
 * `defm left bleblactrl 3`
 * `defm right bleblactrl 4`
* Ask for motor positions: `wm top bottom left right`
* Determine direction with relative move and adjust sign:
 * `bottom.sign = -1`
 * `left.sign = -1`
* Change offset by using the `set_user_pos` macro e.g. `set_user_pos top 0`
* Open and close gap by moving physical motors
* Move offset by moving the physical motors

### Pseudomotor basics - Slit

* `Slit` controller - Sardana's standard slit controller
* Physical axes = motors that will be moved
* Pseudo axes = virtual axes consisting of physical axes
* `pseudo_motor_roles` and `motor_roles` class members defines number of pseudo and physical axes used by the controller
* Instantiate vertical slits controller: `defctrl Slit vertctrl sl2t=top sl2b=bottom Gap=gapvert Offset=offsetvert`
* Instantiate horizontal slits controller: `defctrl Slit horctrl sl2t=right sl2b=left Gap=gaphor Offset=offsethor`

### Inside Slit controller

* `CalcPhysical` method, for calculating physical axes positions based on pseudo axes position
 * Calculate half_gap
 * top = offset + half_gap 
 * Bottom would be offset - half_gap if directions were common
 * Since the directions are opposit bottom = half_gap - offset
* `CalcPseudo` method, for calculating pseudo axes positions based on physical ones
 * gap = bottom + top
 * offset = top - half_gap